In [10]:
import pandas as pd
import pickle
import numpy as np

In [11]:
class CategoricalEncoder:
    
    """
    It also changes the data type to int
    """
    
    def __init__(self, cols, method = 'classic'):
        self.COLUMNS = cols
        self.METHOD = method
        self.encoder = {}
        self.inverse_encoder = {}
        
    def fit(self, X, y=None):
        X = X.copy()
        if self.METHOD == 'classic':
            for col in self.COLUMNS:
                self.encoder[col] = {}
                self.inverse_encoder[col] = {}
                unique_values = X[col].unique()
                for i, val in enumerate(unique_values):
                    self.encoder[col][val] = i
                    self.inverse_encoder[col][i] = val
                    
        if self.METHOD == 'mean':
            for col in self.COLUMNS:
                self.encoder[col] = {}
                self.inverse_encoder[col] = {}
                unique_values = X[col].unique()
                for val in unique_values:
                    tmp = X[col][X[col] == val]
                    mean = y.loc[tmp.index, 'P80'].mean()
                    self.encoder[col][val] = mean
                    self.inverse_encoder[col][mean] = val
                self.encoder['NaN'] = y['P80'].mean()
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.COLUMNS:
            X[col] = X[col].map(self.encoder[col]).fillna(self.encoder['NaN'])
        return X
    
    def inverse_transform(self, X):
        X = X.copy()
        for col in self.COLUMNS:
            X[col] = X[col].replace(self.inverse_encoder[col])
        return X
    

In [12]:
with open('models/CONF.pickle', 'rb') as handle:
    CONF = pickle.load(handle)
    
with open('models/encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [13]:
CATEGORICAL_COLS = ['Fase', 'Tipo de tronadura', 'Tipo Material', 'M', 'Dominio Estructural', 'Tipo Explosivo']
NUMERICAL_COLS = ['Banco', 'Diámetro', 'Fc', 'Cota', 'B', 'S', 'tiempo_1', 'tiempo_2']
TARGET_COLS = ['P10', 'P20', 'P30', 'P40', 'P50', 'P60', 'P70', 'P80', 'P90', 'P100']

In [17]:
df_stage3 = pd.read_excel('data/Datos_Entregable3_Hackathon.xlsx', header=2)

In [16]:
df_historic = pd.read_csv('data/data_fixed.csv', sep = ';')
df_historic = df_historic[df_historic['Tipo Explosivo'] != 'M']
df_historic = df_historic.dropna()

In [ ]:
"""restricciones:
    -100 < FC < 1000
    -tiempo_1 < tiempo_2
    -Diametro, Fraccion, dist min, dist max
    12.25 == 1/4 -> 5.5 - 15 m
    10.625 == 10 5/8 -> 5.5 - 15 m
    9.875 == 9 7/8 -> 5 - 15m
    7.875 == 7 7/8 -> 4 - 15m
    6.5 == 6 1/2 -> 4 - 15m
    6 == 6 -> 3.6 - 15 
    
    25*D_p<T<B/2 ou S/2
    
"""

In [ ]:
"""
Diámetro - BxS - Tiempo entre Pozos Filas ms - Fc - Tipo Explosivo
"""

In [24]:
rho_explosivo = {
"A": 0.77,
"B": 1.34,
"C": 1.32,
"D": 1.00,
"E": 1.32,
"F": 1.30,
"G": 1.00,
"H": 1.00,
"I": 1.20,
"J": 1.32,
"K": 1.32,
"L": 1.32
}

rho_roca = {
"L1": 2.53,
"L2": 2.53,
"L3": 2.65,
"L4": 2.65,
"L5": 2.49,
"L6": 2.49,
"L7": 2.65,
"L8": 2.74
}

In [ ]:
#se debe convertir la pulgada a metros (dividir por 39.370)
def gr_explosivo(D_p,T,rho_explosivo):
    return (3.14 * (D_p/39.370)^2) / 4  *( 16 - T ) * rho_explosivo

def ton_carga(B,S,rho_roca):
    return B*S*15* rho_roca

In [ ]:
FC=list(range(100,1001))

In [ ]:
diametro=[6,6.5,7.875,9.875,10.625,12.25]

In [ ]:
B=list(np.arange(3.5,15.1,0.1))
B = [ round(elem, 2) for elem in B ]

S=list(np.arange(3.5,15.1,0.1))
S = [ round(elem, 2) for elem in S ]

In [ ]:
tiempo_1=list(range(0,201))
tiempo_2=list(range(0,201))

In [20]:
tipo_explosivo=['A','B','C','D','E','F','G','H','I','J','K','L']

In [ ]:
for row in df_stage3:
    for j in diametro:
        for k in B:
            for l in S:
                for m in tiempo_1:
                    for n in tiempo_2:
                        for o in tipo_explosivo:
                            if m>n: # condiciones de borde para tiempo
                                continue
                            elif (j == 6     and ( (k<3.5 or k>=15) or (l<3.5 or l>=15) ) ):  # condiciones de borde para B y S
                                continue
                            elif (j == 6.5   and ( (k<4 or k>=15)   or (l<4   or l>=15) ) ):  # condiciones de borde para B y S
                                continue
                            elif (j == 7.875 and ( (k<4 or k>=15)   or (l<4   or l>=15) ) ):  # condiciones de borde para B y S
                                continue
                            elif (j == 9.875 and ( (k<5 or k>=15)   or (l<5   or l>=15) ) ):  # condiciones de borde para B y S
                                continue 
                            elif (j == 10.625and ( (k<5.5 or k>=15) or (l<5.5 or l>=15) ) ):  # condiciones de borde para B y S
                                continue
                            elif (j == 12.25 and ( (k<5.5 or k>=15) or (l<5.5 or l>=15) ) ):  # condiciones de borde para B y S
                                continue
                            else: # cumple con las condiciones minimas de B y S y tiempo
                                if 25*j/39.370>T or T>k/2 or T>l/2: 
                                    continue
                                else: #cumple con las condiciones del TACO
                                    if #TODO arreglar el FC
                                        print ("diametro: "+ str(j) )
                                        print ("B: " + str(k))
                                        print ("S: "+ str(l))
                                        print ("tiempo_1: "+ str(m))
                                        print ("tiempo_2: "+ str(n))
                                        print ("Tipo explosivo: " + str(o))
                            

In [22]:
for index,row in df_stage3.iterrows():
    print (row)

Fase                       STAGE 4
Banco                         2795
Tipo de tronadura      TRONADURA 2
Tipo Material               ROCA 1
M                               L4
Dominio Estructural          Norte
P10                        1.06299
P20                        1.41732
P30                        1.85039
P40                        2.40158
P50                        3.18898
P60                        3.97638
P70                        4.88189
P80                        6.02363
P90                        7.79528
P100                       14.5669
Este                         58999
Norte                        90277
Cota                          2795
Name: 0, dtype: object
Fase                       STAGE 3
Banco                         3815
Tipo de tronadura      TRONADURA 2
Tipo Material               LASTRE
M                               L8
Dominio Estructural          Norte
P10                        1.22047
P20                         1.5748
P30                        1.889